# Optimizer Tests

This notebook implements static environments to test optimizers and ensure that they are correctly implemented and learn a fixed function, when mechanisms do not change over time and do not depend on other players.

In [ ]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

In [ ]:
import warnings
import pytest
import torch
from bnelearn.strategy import NeuralNetStrategy
from bnelearn.mechanism import StaticMechanism
from bnelearn.bidder import Bidder
from bnelearn.optimizer import ES, SimpleReinforce
from bnelearn.environment import AuctionEnvironment

In [ ]:
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

# set up matplotlib
is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
# Shared objects
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

SIZE_HIDDEN_LAYER = 20
input_length = 1

u_lo = 0
u_hi = 10

def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=1)

mechanism = StaticMechanism(cuda=cuda)

In [ ]:
run_dir = 'tests'
run_name = 'test_optim_m6'

BATCH_SIZE = 2**18
epoch = 5000
plot_epoch = 100
LEARNING_RATE = 1e-1
MOMENTUM = 0.8
lr_decay = True
lr_decay_every = 250
lr_decay_factor = 0.8

sigma = .1 #ES noise parameter
n_perturbations = 64


logdir = os.path.join(root_path, 'notebooks', run_dir , run_name)

In [ ]:
def log_hyperparams(writer, e):
    writer.add_scalar('hyperparams/batch_size', BATCH_SIZE, e)
    writer.add_scalar('hyperparams/size_hidden_layer', SIZE_HIDDEN_LAYER)
    writer.add_scalar('hyperparams/learning_rate', LEARNING_RATE, e)
    writer.add_scalar('hyperparams/momentum', MOMENTUM, e)
    writer.add_scalar('hyperparams/sigma', sigma, e)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations, e)

def optimal_bid(v):
    return v

def plot_bid_function(fig, v,b, writer=None, e=None, plot_points=100):
    
    # subsample points and plot
    v = v.detach().cpu().numpy()[:plot_points]
    b= b.detach().cpu().numpy()[:plot_points]
    
    optimal = optimal_bid(v)
    fig = plt.gcf()
    plt.cla()
    plt.plot(v,b, 'o', v, optimal, 'r-')
    if is_ipython:
        #display.clear_output(wait=True)
        display.display(plt.gcf())
    if writer:
        writer.add_figure('eval/bid_function', fig, e)  

In [ ]:
model = NeuralNetStrategy(input_length, size_hidden_layer = SIZE_HIDDEN_LAYER, requires_grad=False).to(device)
 #bidder = strat_to_bidder(model, BATCH_SIZE)
env = AuctionEnvironment(
    mechanism,
    agents = [],
    strategy_to_bidder_closure=strat_to_bidder,
    max_env_size=1,
    batch_size = BATCH_SIZE,
    n_players=1
    )

optimizer = ES(
        model=model,
        environment = env,
        env_type = 'fixed',
        lr = LEARNING_RATE,
        momentum = MOMENTUM,
        sigma=sigma,
        n_perturbations=n_perturbations
        )

torch.cuda.empty_cache()

writer = SummaryWriter(logdir)
log_hyperparams(writer, 0)
fig = plt.figure()


for e in range(epoch+1):
    # lr decay?
    if lr_decay and e % lr_decay_every == 0 and e > 0:
        LEARNING_RATE = LEARNING_RATE * lr_decay_factor
        log_hyperparams(writer, e)
        for param_group in optimizer.param_groups:
            param_group['lr'] = LEARNING_RATE
        
    # always: do optimizer step
    utility = -optimizer.step()
    writer.add_scalar('eval/utility', utility, e)
    
    if e % plot_epoch == 0:
        # plot current function output
        bidder = strat_to_bidder(model, BATCH_SIZE)
        bidder.draw_valuations_()
        v = bidder.valuations
        b = bidder.get_action()
        share = b.mean()/v.mean()
        diff = (b-v).mean()
        writer.add_scalar('eval/share', share, e)
        writer.add_scalar('eval/diff', diff, e) 
        
        
        print("Epoch {}: \ttotal share: {:2f}, diff: {:2f}, \tutility: {:2f}".format(e, share, diff, utility))
        plot_bid_function(fig, v,b,writer,e)
        
        
        
        # first step: write model
        if e==0:
            writer.add_graph(model, bidder.valuations) 
        
torch.cuda.empty_cache()
writer.close()